#### Cây quyết định sử dụng Gini

In [2]:
import numpy as np

def gini_index(groups, classes):
    """ Tính toán chỉ số Gini của một tập hợp """
    total_samples = float(sum([len(group) for group in groups]))
    gini = 0.0
    for group in groups:
        size = float(len(group))
        if size == 0:
            continue
        score = sum([(row[-1] == c) for row in group for c in classes])/ size
        gini += (1.0 - sum([score ** 2 for c in classes])) * (size/ total_samples)
    return gini

# Ví dụ dữ liệu
dataset = [[2.8, 'Yes'],
            [1.2, 'No'],
            [3.6, 'Yes'],
            [4.5, 'No'],
            [5.1, 'Yes']]

# Chia tập dữ liệu theo một giá trị ngưỡng
def split_data(dataset, feature_index, threshold):
    left = [row for row in dataset if row[feature_index] < threshold]
    right = [row for row in dataset if row[feature_index] >= threshold]
    return left, right

# Ví dụ tính Gini cho một cách chia dữ liệu
groups = split_data(dataset, 0, 3.0)
classes = ['Yes', 'No']
gini = gini_index(groups, classes)
print (f'Gini Index: {gini:.4f}')

Gini Index: -1.0000


#### Cây quyết định sử dụng OOP - Lớp TreeNode - Biểu diễn một nút trong cây

In [3]:
class TreeNode:
    def __init__(self, feature_index = None, threshold = None, left = None, right = None, label = None):
    # Khởi tạo một nút trong cây quyết định.
    # - feature_index: Chỉ số thuộc tính được chọn để chia.
    # - threshold: Ngưỡng giá trị để phân chia dữ liệu.
    # - left: Nhánh trái của cây.
    # - right: Nhánh phải của cây.
    # - label: Nhãn dự đoán nếu là nút lá.

        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.label = label

#### Cây quyết định sử dụng OOP - Lớp DecisionTree - Xây dựng cây quyết định

In [4]:
import numpy as np

class DecisionTree:
    def __init__(self, max_depth = 3):
    # Khởi tạo cây quyết định với độ sâu tối đa.
        self.max_depth = max_depth
        self.root = None

    def gini_index(self, groups, classes):
    # Tính chỉ số Gini cho một tập hợp.
        total_samples = sum([len(group) for group in groups])
        gini = 0.0
        for group in groups:
            size = len(group)
            if size == 0:
                continue
            score = 0.0
            for class_val in classes:
                proportion = [row[-1] for row in group].count(class_val)/ size
                score += proportion ** 2
            gini += (1.0 - score) * (size/ total_samples)
        return gini

    def split_data(self, dataset, feature_index, threshold):
        # Chia tập dữ liệu dựa trên một thuộc tính và ngưỡng giá trị.
        left = [row for row in dataset if row[feature_index] < threshold]
        right = [row for row in dataset if row[feature_index] >= threshold]
        return left, right

    def best_split(self, dataset):
        # Tìm thuộc tính tốt nhất để chia tập dữ liệu.
        class_values = list(set(row[-1] for row in dataset))
        best_index, best_threshold, best_score, best_groups = None, None, float('inf'), None

        for index in range(len(dataset[0]) - 1):
            for row in dataset:
                groups = self.split_data(dataset, index, row[index])
                gini = self.gini_index(groups, class_values)
                if gini < best_score:
                    best_index, best_threshold, best_score, best_groups = index, row[index], gini, groups
        return best_index, best_threshold, best_groups

    def build_tree(self, dataset, depth =0):
        # Xây dựng cây quyết định đệ quy.
        class_values = [row[-1] for row in dataset]

        # Điều kiện dừng: Nếu chỉ có một lớp hoặc đạt đến độ sâu tối đa
        if len(set(class_values)) == 1 or depth >= self.max_depth:
            return TreeNode(label = max(set(class_values), key = class_values.count))

        # Tìm thuộc tính và giá trị ngưỡng tốt nhất để chia dữ liệu
        feature_index, threshold, (left, right) = self.best_split(dataset)
        # Nếu không thể chia tiếp, tạo nút lá
        if not left or not right:
            return TreeNode(label = max(set(class_values), key = class_values.count))

        # Xây dựng nhánh trái và nhánh phải
        left_node = self.build_tree(left, depth + 1)
        right_node = self.build_tree(right, depth + 1)

        return TreeNode(feature_index, threshold, left_node, right_node)

    def fit(self, dataset):
    # Huấn luyện cây quyết định bằng cách xây dựng cây từ dữ liệu đầu vào.
        self.root = self.build_tree(dataset)

    def print_tree(self, node = None, depth = 0):
    # In ra cây quy ết định theo dạng phân cấp.
        if node is None:
            node = self.root

        if node.label is not None:
            print(f"{' ' * depth} [Leaf] Label: {node.label}")
        else:
            print(f"{' ' * depth} [Node] Feature {node.feature_index} <= {node.threshold}")
            self.print_tree(node.left, depth + 1)
            self.print_tree(node.right, depth + 1)

In [5]:
# Tập dữ liệu đơn giản: [Thuộc tính, Nhãn]
dataset = [[2.8, 'Yes'],
            [1.2, 'No'],
            [3.6, 'Yes'],
            [4.5, 'No'],
            [5.1, 'Yes']]

# Khởi tạo và huấn luyện cây quy ết định
tree = DecisionTree(max_depth =3)
tree.fit(dataset)

# In ra cây quyết định
print("Cây quyết định được xây dựng:")
tree.print_tree()

Cây quyết định được xây dựng:
 [Node] Feature 0 <= 2.8
  [Leaf] Label: No
  [Node] Feature 0 <= 4.5
   [Leaf] Label: Yes
   [Node] Feature 0 <= 5.1
    [Leaf] Label: No
    [Leaf] Label: Yes


#### Bài 1

In [6]:
dataset = [[50, 'Yes'],
            [20, 'No'],
            [30, 'No'],
            [70, 'Yes'],
            [40, 'No'],
            [60, 'Yes']]

def cal_gini(data):
    total = len(data)
    yes_count = sum(1 for row in data if row[-1] == 'Yes')
    no_count = total - yes_count
    
    p_yes = yes_count / total
    p_no = no_count / total 

    gini = 1 - (p_yes**2 + p_no**2)
    return gini

print(cal_gini(dataset))

0.5


#### Bài 2

In [7]:
data_extended = [
    [50, 25, 700, 'Yes'],  # [Luong, Tuoi, Diem_tin_dung, Ket_qua]
    [20, 30, 500, 'No'],
    [30, 35, 600, 'No'],
    [70, 40, 800, 'Yes'],
    [40, 45, 550, 'No'],
    [60, 50, 750, 'Yes']
]

In [8]:
print(cal_gini(data_extended))

0.5


In [9]:
X = np.array([[row[0], row[1], row[2]] for row in data_extended])
y = np.array([row[3] for row in data_extended])

dataset = [list(x) + [y_val] for x, y_val in zip(X, y)]

tree = DecisionTree(max_depth = 3)
tree.fit(dataset)
tree.print_tree()

 [Node] Feature 0 <= 50
  [Leaf] Label: No
  [Leaf] Label: Yes


In [15]:
import numpy as np

class DecisionTree:
    def __init__(self, max_depth = 3):
    # Khởi tạo cây quyết định với độ sâu tối đa.
        self.max_depth = max_depth
        self.root = None
        self.feature_names = None  # Thêm tên các thuộc tính

    def gini_index(self, groups, classes):
    # Tính chỉ số Gini cho một tập hợp.
        total_samples = sum([len(group) for group in groups])
        gini = 0.0
        for group in groups:
            size = len(group)
            if size == 0:
                continue
            score = 0.0
            for class_val in classes:
                proportion = [row[-1] for row in group].count(class_val)/ size
                score += proportion ** 2
            gini += (1.0 - score) * (size/ total_samples)
        return gini

    def split_data(self, dataset, feature_index, threshold):
        # Chia tập dữ liệu dựa trên một thuộc tính và ngưỡng giá trị.
        left = [row for row in dataset if row[feature_index] < threshold]
        right = [row for row in dataset if row[feature_index] >= threshold]
        return left, right

    def best_split(self, dataset):
        # Tìm thuộc tính tốt nhất để chia tập dữ liệu.
        class_values = list(set(row[-1] for row in dataset))
        best_index, best_threshold, best_score, best_groups = None, None, float('inf'), None

        for index in range(len(dataset[0]) - 1):
            for row in dataset:
                groups = self.split_data(dataset, index, row[index])
                gini = self.gini_index(groups, class_values)
                if gini < best_score:
                    best_index, best_threshold, best_score, best_groups = index, row[index], gini, groups
        return best_index, best_threshold, best_groups

    def build_tree(self, dataset, depth =0):
        # Xây dựng cây quyết định đệ quy.
        class_values = [row[-1] for row in dataset]

        # Điều kiện dừng: Nếu chỉ có một lớp hoặc đạt đến độ sâu tối đa
        if len(set(class_values)) == 1 or depth >= self.max_depth:
            return TreeNode(label = max(set(class_values), key = class_values.count))

        # Tìm thuộc tính và giá trị ngưỡng tốt nhất để chia dữ liệu
        feature_index, threshold, (left, right) = self.best_split(dataset)
        # Nếu không thể chia tiếp, tạo nút lá
        if not left or not right:
            return TreeNode(label = max(set(class_values), key = class_values.count))

        # Xây dựng nhánh trái và nhánh phải
        left_node = self.build_tree(left, depth + 1)
        right_node = self.build_tree(right, depth + 1)

        return TreeNode(feature_index, threshold, left_node, right_node)

    def fit(self, dataset, feature_names=None):
        # Thêm tên các thuộc tính khi huấn luyện
        self.feature_names = feature_names if feature_names else [f"Feature_{i}" for i in range(len(dataset[0])-1)]
        self.root = self.build_tree(dataset)
    
    def print_tree(self, node=None, depth=0, is_last=True, prefix=""):
        if node is None:
            node = self.root

        # Tạo các ký tự để vẽ cây
        indent = "    " * depth  # Tăng khoảng cách thụt lề
        branch = "└── " if is_last else "├── "
        
        # In node hiện tại
        if node.label is not None:
            print(f"{prefix}{branch}Kết quả: {node.label}")
        else:
            feature_name = self.feature_names[node.feature_index]
            print(f"{prefix}{branch}{feature_name} ≤ {node.threshold}")
            
            # Chuẩn bị prefix cho các node con
            new_prefix = prefix + ("    " if is_last else "│   ")
            
            # In node con trái
            self.print_tree(node.left, depth + 1, False, new_prefix)
            # In node con phải
            self.print_tree(node.right, depth + 1, True, new_prefix)


In [16]:
# Dữ liệu mở rộng với tên các thuộc tính
data_extended = [
    [50, 25, 700, 'Yes'],  # [Luong, Tuoi, Diem_tin_dung, Ket_qua]
    [20, 30, 500, 'No'],
    [30, 35, 600, 'No'],
    [70, 40, 800, 'Yes'],
    [40, 45, 550, 'No'],
    [60, 50, 750, 'Yes']
]

# Tên các thuộc tính
feature_names = ['Lương (triệu)', 'Tuổi', 'Điểm tín dụng']

# Khởi tạo và huấn luyện cây quyết định
tree = DecisionTree(max_depth=3)
tree.fit(data_extended, feature_names)

# In ra cây quyết định với định dạng dễ đọc
print("Cây quyết định cho vay:")
print("└─ Root")
tree.print_tree()

Cây quyết định cho vay:
└─ Root
└── Lương (triệu) ≤ 50
    ├── Kết quả: No
    └── Kết quả: Yes
